In [1]:
import os
import warnings

from langchain_core.output_parsers import StrOutputParser

warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Load the data
df = pd.read_csv('Supplier_data.csv', index_col=0)

In [3]:
df.head()

,supplier_name,price,price_weight,sust_weight,sust_criterion1,sust_criterion1_weight,sust_criterion2,sust_criterion2_weight,Eignungskriterien,Nachweis der Teilnahmebedingungen Selbstdeklaration,...,Akzeptanz des Rahmenvertragsentwurfs,Technische Spezifikation,Lieferfrist,NachhaltigkeitsKriterien,EcoVadis Bewertungsplattform,Umweltmanagement,Qulitätsmanagement,Nachhaltigkeitsbericht,Treibhausausstoss beim Produkt/ der Dienstleistung,Reduktion Treibhausgase in Unternehmung
<anonymous>,,,,,,,,,,,,,,,,,,,,,
1,Swisscom,200,0.8,0.2,True,0.5,True,0.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Create a useful feature column list for analysis
col_name = df.columns

train = df[col_name]
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, 1 to 1
Data columns (total 24 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   supplier_name                                        1 non-null      object 
 1   price                                                1 non-null      int64  
 2   price_weight                                         1 non-null      float64
 3   sust_weight                                          1 non-null      float64
 4   sust_criterion1                                      1 non-null      bool   
 5   sust_criterion1_weight                               1 non-null      float64
 6   sust_criterion2                                      1 non-null      bool   
 7   sust_criterion2_weight                               1 non-null      float64
 8   Eignungskriterien                                    0 non-null      float64
 9

In [5]:
!pip install pyahp

In [6]:
from pyahp import parse
import pandas as pd
import json
import numpy as np

# Sample data setup
data = {
    'supplier_name': ['Swisscom', 'Sunrise', 'Salt'],
    'price': [200, 150, 180],
    'price_weight': [0.8, 0.8, 0.8],
    'sust_weight': [0.2, 0.2, 0.2],
    'sust_criterion1': [True, False, True],
    'sust_criterion1_weight': [0.5, 0.5, 0.5],
    'sust_criterion2': [True, True, False],
    'sust_criterion2_weight': [0.5, 0.5, 0.5]
}

df = pd.DataFrame(data)

# Define AHP model with adjusted pairwise comparisons for Price
ahp_model = {
    "name": "Supplier Selection",
    "method": "geometric",
    "criteria": ["Price", "Sustainability"],
    "subCriteria": {
        "Sustainability": ["Criterion1", "Criterion2"]
    },
    "alternatives": df['supplier_name'].tolist(),
    "preferenceMatrices": {
        "criteria": [
            [1, df['price_weight'][0] / df['sust_weight'][0]], 
            [df['sust_weight'][0] / df['price_weight'][0], 1]
        ],
        
        # Adjusted Price matrix based on relative prices
        "alternatives:Price": [
            [1, 200 / 150, 200 / 180],    # Swisscom compared to Sunrise and Salt
            [150 / 200, 1, 150 / 180],    # Sunrise compared to Swisscom and Salt
            [180 / 200, 180 / 150, 1]     # Salt compared to Swisscom and Sunrise
        ],
        
        # Sub-criteria for Sustainability (placeholders, adjust based on your criteria)
        "subCriteria:Sustainability": [
            [1, df['sust_criterion1_weight'][0] / df['sust_criterion2_weight'][0]], 
            [df['sust_criterion2_weight'][0] / df['sust_criterion1_weight'][0], 1]
        ],
        
        # Placeholder matrices for Sustainability sub-criteria
        "alternatives:Criterion1": [
            [1, 1, 1],
            [1, 1, 1],
            [1, 1, 1]
        ],
        "alternatives:Criterion2": [
            [1, 1, 1],
            [1, 1, 1],
            [1, 1, 1]
        ],
        
        # Placeholder matrix for Sustainability alternatives
        "alternatives:Sustainability": [
            [1, 1, 1],
            [1, 1, 1],
            [1, 1, 1]
        ]
    }
}

# Run the AHP model and convert results to a list for serialization
results = parse(ahp_model).get_priorities()

# Print the AHP results in JSON format
print("AHP Results:\n", results)


AHP Results:
 [3.28  2.71  3.052]


In [7]:
!pip install ahpy
!pip install numpy==1.26.0 
## downgrade numpy to 1.26.0 otherwise the next line doesn't work

In [44]:
import pandas as pd

data = {
    'supplier_name': ['Swisscom', 'Sunrise', 'Salt'],
    'price': [150, 150, 150],
    'sust_criterion1': [True, True, True],
    'sust_criterion2': [True, True, False],
}


df = pd.DataFrame(data)

def generate_pairwise_comparisons(items, comparison_matrix):
    comparisons = {}
    for i, item1 in enumerate(items):
        for j, item2 in enumerate(items):
            if i < j:
                comparisons[(item1, item2)] = comparison_matrix[i][j]
                comparisons[(item2, item1)] = 1 / comparison_matrix[i][j]
    return comparisons

In [1]:
import pandas as pd
import ahpy

# Your data with dynamic subcriteria
data = {
    # Enter the supplier names
    'supplier_name': ['Swisscom', 'Sunrise', 'Salt'],
    # Enter the price values for each supplier
    'price': [5344.0, 342.0, 172911.0], # Less is better
    # 2000 points in total for sustainability criteria
    'sust_EcoVadis': [266.67,266.67,400.0], # More is better
    'sust_Environmental_management': [300.0, 300.0, 300.0], # More is better
    'sust_Quality_management': [300.0, 300.0, 300.0], # More is better
    'sust_Sustainability_report': [400.0, 400.0, 400.0], # More is better
    'sust_Carbon_footprint:Quantification_at_product_level': [300.0, 300.0, 300.0], # More is better
    'sust_Carbon_footprint:Reduction_in_company': [200.0, 200.0, 200.0], # More is better
    # 'sust_Solar_energy_usage': [50.0, 10.0, 200.0], # More is better
    # Must criteria are not considered in AHP but are displayed for the final decision
    'must_Confirmation_of_technical_specification': [False, True ,True],
    'must_Social_insurance_contributions': [True, False, True],
    # Add more criteria as needed
}

# Convert data to DataFrame
df = pd.DataFrame(data)

# Extract supplier names
suppliers = df['supplier_name'].tolist()

# Identify subcriteria columns dynamically
# Assuming all columns starting with 'sust_' are sustainability subcriteria
subcriteria_columns = [col for col in df.columns if col.startswith('sust_')]
subcriteria = [sc.replace('sust_', '') for sc in subcriteria_columns]
# Assuming all columns starting with 'must_' are must criteria
mustcriteria_columns = [col for col in df.columns if col.startswith('must_')]

# Fixed weights for criteria
price_weight = 0.8
sustainability_weight = 0.2

# Criteria Comparison
importance_ratio = price_weight / sustainability_weight  # Should be 4
criteria_comparisons = {
    ('Price', 'Sustainability'): importance_ratio,
}
criteria = ahpy.Compare('Criteria', criteria_comparisons, precision=3, random_index='saaty')

# Subcriteria Comparisons (assuming equal importance)
sustainability_comparisons = {}
for i, sc1 in enumerate(subcriteria):
    for sc2 in subcriteria[i+1:]:
        sustainability_comparisons[(sc1, sc2)] = 1  # Equally important
sustainability = ahpy.Compare('Sustainability', sustainability_comparisons, precision=3, random_index='saaty')

# Alternatives Comparison for Price
if df['price'].nunique() == 1:
    price_comparisons = {
        (s1, s2): 1 for i, s1 in enumerate(suppliers) for s2 in suppliers[i+1:]
    }
else:
    price_comparisons = {}
    for i, s1 in enumerate(suppliers):
        for s2 in suppliers[i+1:]:
            p1 = df.loc[df['supplier_name'] == s1, 'price'].values[0]
            p2 = df.loc[df['supplier_name'] == s2, 'price'].values[0]
            comparison_value = p2 / p1
            price_comparisons[(s1, s2)] = comparison_value
price = ahpy.Compare('Price', price_comparisons, precision=3, random_index='saaty')
# Alternatives Comparisons for Subcriteria
subcriteria_compares = {}
for sc in subcriteria_columns:
    sc_name = sc.replace('sust_', '')  # For clarity
    comparisons = {}
    for i, s1 in enumerate(suppliers):
        for s2 in suppliers[i+1:]:
            val_s1 = df.loc[df['supplier_name'] == s1, sc].values[0]
            val_s2 = df.loc[df['supplier_name'] == s2, sc].values[0]
            if val_s1 == val_s2:
                comparison_value = 1
            elif val_s1 and not val_s2:
                comparison_value = 9
            elif not val_s1 and val_s2:
                comparison_value = 1/9
            else:
                comparison_value = 1
            comparisons[(s1, s2)] = comparison_value
    subcriteria_compares[sc_name] = ahpy.Compare(sc_name, comparisons, precision=3, random_index='saaty')

# Build the AHP hierarchy
sustainability.add_children(list(subcriteria_compares.values()))
criteria.add_children([price, sustainability])

# Compute the final priorities
overall_priorities = criteria.target_weights

# Collect must criteria status for each supplier
must_criteria_status = {"Mandatory Criteria Status": {}}
for supplier in suppliers:
    status = {}
    for mc in mustcriteria_columns:
        value = df.loc[df['supplier_name'] == supplier, mc].values[0]
        status[mc.replace('must_', '')] = value
    must_criteria_status["Mandatory Criteria Status"][supplier] = status

# Print must criteria status
must_criteria_report = "\nMust Criteria Status:\n"
for supplier, status in must_criteria_status["Mandatory Criteria Status"].items():
    criteria_met = []
    criteria_not_met = []
    for criterion, value in status.items():
        if not value:
            criteria_not_met.append(criterion)
        else:
            criteria_met.append(criterion)
    if criteria_met:
        must_criteria_report += f"{supplier} meets the must criteria: {', '.join(criteria_met) if criteria_met else 'None'}\n"
    if criteria_not_met:
        must_criteria_report += f"{supplier} does NOT meet the must criteria: {', '.join(criteria_not_met) if criteria_not_met else 'None'}\n"

print("\nOverall priorities:")
for supplier, weight in overall_priorities.items():
    print(f"{supplier}: {weight:.3f}")

# Simplified report
print(must_criteria_report)
print("\nReport:")
# print(must_criteria_status)
print(criteria.report(show=True, complete=False))



Overall priorities:
Sunrise: 0.817
Swisscom: 0.115
Salt: 0.068

Must Criteria Status:
Swisscom meets the must criteria: Social_insurance_contributions
Swisscom does NOT meet the must criteria: Confirmation_of_technical_specification
Sunrise meets the must criteria: Confirmation_of_technical_specification
Sunrise does NOT meet the must criteria: Social_insurance_contributions
Salt meets the must criteria: Confirmation_of_technical_specification, Social_insurance_contributions


Report:
{
    "name": "Criteria",
    "global_weight": 1.0,
    "local_weight": 1.0,
    "target_weights": {
        "Sunrise": 0.817,
        "Swisscom": 0.115,
        "Salt": 0.068
    },
    "elements": {
        "global_weights": {
            "Price": 0.8,
            "Sustainability": 0.2
        },
        "local_weights": {
            "Price": 0.8,
            "Sustainability": 0.2
        },
        "consistency_ratio": 0.0
    }
}
{'name': 'Criteria', 'global_weight': 1.0, 'local_weight': 1.0, 'targe

In [2]:
# Detailed report for the AI Agent
print("\nDetailed report:")
print(must_criteria_status)
criteria.report(show=True, complete=True)


Detailed report:
{'Mandatory Criteria Status': {'Swisscom': {'Confirmation_of_technical_specification': False, 'Social_insurance_contributions': True}, 'Sunrise': {'Confirmation_of_technical_specification': True, 'Social_insurance_contributions': False}, 'Salt': {'Confirmation_of_technical_specification': True, 'Social_insurance_contributions': True}}}
{
    "Criteria": {
        "name": "Criteria",
        "global_weight": 1.0,
        "local_weight": 1.0,
        "target_weights": {
            "Sunrise": 0.817,
            "Swisscom": 0.115,
            "Salt": 0.068
        },
        "elements": {
            "global_weights": {
                "Price": 0.8,
                "Sustainability": 0.2
            },
            "local_weights": {
                "Price": 0.8,
                "Sustainability": 0.2
            },
            "consistency_ratio": 0.0
        }
    },
    "Price": {
        "name": "Price",
        "global_weight": 0.8,
        "local_weight": 0.8,
       

{'Criteria': {'name': 'Criteria',
  'global_weight': 1.0,
  'local_weight': 1.0,
  'target_weights': {'Sunrise': 0.817, 'Swisscom': 0.115, 'Salt': 0.068},
  'elements': {'global_weights': {'Price': 0.8, 'Sustainability': 0.2},
   'local_weights': {'Price': 0.8, 'Sustainability': 0.2},
   'consistency_ratio': 0.0}},
 'Price': {'name': 'Price',
  'global_weight': 0.8,
  'local_weight': 0.8,
  'target_weights': None,
  'elements': {'global_weights': {'Sunrise': 0.75,
    'Swisscom': 0.048,
    'Salt': 0.002},
   'local_weights': {'Sunrise': 0.938, 'Swisscom': 0.06, 'Salt': 0.002},
   'consistency_ratio': 0.0}},
 'Sustainability': {'name': 'Sustainability',
  'global_weight': 0.2,
  'local_weight': 0.2,
  'target_weights': None,
  'elements': {'global_weights': {'EcoVadis': 0.033,
    'Environmental_management': 0.033,
    'Quality_management': 0.033,
    'Sustainability_report': 0.033,
    'Carbon_footprint:Quantification_at_product_level': 0.033,
    'Carbon_footprint:Reduction_in_compan

In [8]:
# LLM
import os
from dotenv import load_dotenv, find_dotenv

model = 'llama-3.1-70b-versatile'
temperature = 0
max_new_tokens = 1024

load_dotenv(find_dotenv())
global GROQ_API_KEY
if os.getenv("GROQ_API_KEY") is not None:
    GROQ_API_KEY = os.getenv("GROQ_API_KEY")
else:
    raise ValueError("Please set the GROQ_API_KEY environment variable.")

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser

llm = ChatGroq(
    model=model,
    temperature=temperature,
    # max_tokens=_max_new_tokens,
    timeout=None,
    max_retries=5,
    api_key=GROQ_API_KEY,
    # other params...
)
# Append Must criteria to the report
report = must_criteria_status | criteria.report(show=False, complete=True)
prompt = ChatPromptTemplate.from_template(
    """You are a supply chain management expert. Your task is to provide a recommendation of a supplier based on the results of the AHP report. 
    Consider all suppliers, the final decision belongs to the human.
    AHP report: {report}
    """).format(report=report)

chain =  llm | StrOutputParser()

for token in chain.stream(prompt):
    print(token, end='', flush=True)
    

Based on the AHP report provided, I will analyze the results and provide a recommendation for a supplier.

**Mandatory Criteria Status:**

* Swisscom: Failed to confirm technical specification, but has social insurance contributions.
* Sunrise: Confirmed technical specification, but lacks social insurance contributions.
* Salt: Confirmed technical specification and has social insurance contributions.

**Criteria Weights:**

* Price: 0.8
* Sustainability: 0.2

**Supplier Weights:**

* Sunrise: 0.817 (Criteria), 0.75 (Price), 0.167 (Sustainability)
* Swisscom: 0.115 (Criteria), 0.048 (Price), 0.333 (Sustainability)
* Salt: 0.068 (Criteria), 0.002 (Price), 0.333 (Sustainability)

Based on the AHP report, Sunrise has the highest weight in the Criteria category (0.817), which is heavily influenced by the Price category (0.75). However, Sunrise lacks social insurance contributions, which is a mandatory criterion.

Salt, on the other hand, has confirmed technical specification and has social 